In [19]:
import pandas as pd
import matplotlib.pyplot as plt


# load in a bunch of potentially useful files
# loading merged log_tpm_file
# merge together log_tpm_df files
log_tpm_df = pd.read_csv('../data/precise_1.0/log_tpm.csv', index_col = 0)
starve_log_tpm = pd.read_csv('../data/validation_data_sets/stationary_phase/cleaned_log_tpm_qc.csv', index_col = 0)
to_blank_inds = list(set(log_tpm_df.index) - set(starve_log_tpm.index))
# need to create zero rows for missing values
zeros_data = {col : 0 for col in starve_log_tpm.columns}
zeros_df = pd.DataFrame(zeros_data, index = to_blank_inds)
starve_log_tpm = pd.concat([starve_log_tpm, zeros_df])
starve_log_tpm = starve_log_tpm.loc[log_tpm_df.index]
log_tpm_df = pd.concat([starve_log_tpm, log_tpm_df], axis = 1)

iM_to_genes = {}
gene_to_iMs = {}
k_to_iM = {}
f = open('../data/precise_1.0/iM_table.csv', 'r')
lines = f.readlines()
f.close()
k_to_iM = {line.split(',')[0] : line.split(',')[1] for line in lines[1:]}
iM_to_regulator = {line.split(',')[1] : line.split(',')[2] for line in lines[1:]}
f = open('../data/precise_1.0/gene_presence_list.csv', 'r')
lines = f.readlines()
f.close()
for line in lines[1:]:
    k = line.split(',')[1]
    iM = k_to_iM[k]
    gene = line.split(',')[2].replace('\n', '')
    if iM not in iM_to_genes:
        iM_to_genes.update({iM : []})
    iM_to_genes[iM].append(gene)
    if gene not in gene_to_iMs:
        gene_to_iMs.update({gene : []})
    gene_to_iMs[gene].append(iM)
    
A = pd.read_csv('../data/precise_1.0/A.csv', index_col = 0).rename(index = {int(k) : v for k, v in k_to_iM.items()})
all_A = pd.read_csv('../data/validation_data_sets/merged_A.csv', index_col = 0)

TRN_df = pd.read_pickle('../data/regulonDB_files/TRN_df.pkl') # TRN_df.loc[regulator][target] = 1 means regulator regulates target

In [9]:
# find iMs
iM_reg_df = pd.DataFrame(index = log_tpm_df.index)
promoters = []
inhibitors = []
for gene in iM_reg_df.index:
    # need to decide if iM is repressor or promoter
    if gene not in gene_to_iMs:
        promoters.append([])
        inhibitors.append([])
        continue # gene not in iModulons, move on
    proms = []
    inhs = []
    for iM in gene_to_iMs[gene]:
        # need to decide if a given iModulon is a promoter or inhibitor of the gene
        
        # need to do this, but here's how I think I tell
        # positive gene weight and positive correlation between regulator and iM = promoter
        # negative gene weight and positive correlation between regulator and iM = inhibitor
        # positive gene weight and negative correlation between regulator and iM = inhibitor
        # negative gene weight and negative correlation between regulator and iM = promoter
        # so I need the regulator name don't I... maybe it's easier to just go through them manually as that'll be quicker than connecting all of these together
        

In [27]:
import numpy as np
np.corrcoef([0, 1, 40, -5, 1.1, 0.1, 100], [0, 10, 5, 20, 1, 5, 100])

array([[1.        , 0.88510586],
       [0.88510586, 1.        ]])

In [18]:
f = open('../data/precise_1.0/iM_table.csv', 'r')
lines = f.readlines()
f.close()
lines[0:5]

['k,name,regulator_readable,function,category,n_genes,exp_var,precision,recall,category_num\n',
 '0,YieP,,Unknown,Regulator Discovery,11,0.00012755418801002616,,,7\n',
 '1,GlpR,GlpR,Glycerol catabolism,Carbon Source Utilization,9,0.0,1.0,1.0,0\n',
 '2,YgbI,,Unknown Function,Regulator Discovery,9,0.0,,,7\n',
 '3,efeU-repair,,Accounts for repair and expression of efeU operon,Genomic Alterations,8,0.0004772038339341256,,,9\n']